# Multi-Agent Deep Research with Pydantic AI

Our multi-agent system consists of four specialized agents:

- **Clarifier Agent**: Understands user intent and refines research questions
- **Research Agent**: Performs systematic exploration across three stages
- **Verifier/Synthesizer Agent**: Verifies that the claims are correct and combines findings into a cohesive article

In this example, we'll follow a pre-defined flow:

1. User asks initial question
2. We clarify the question and intent
3. We conduct research (phases 1, 2, 3)
4. We hand off the research results to generate the final article

In the next lesson, we'll keep it a bit more flexible by introducing the **Orchestrator Agent** - an agent that coordinates the entire workflow.

## Prework

### Data Setup and Indexing

In [1]:
from pathlib import Path

data_folder = Path('../data_cache/youtube_videos/')
data_files = sorted(data_folder.glob("*.txt"))

Process and chunk the transcript files:

In [2]:
import docs
from tqdm.auto import tqdm

documents = []

for f in tqdm(data_files):
    filename = f.name
    video_id, _ = filename.split('.')
    content = f.read_text(encoding='utf-8')
    chunks = docs.sliding_window(content, size=3000, step=1500)

    for chunk in chunks:
        chunk['video_id'] = video_id
        documents.append(chunk)

  0%|          | 0/191 [00:00<?, ?it/s]

Index the documents for search:

In [3]:
from minsearch import Index

index = Index(
    text_fields=["content"],
    keyword_fields=["video_id"]
)

index.fit(documents)


### Search Tool Implementation

Create enhanced search tools with document retrieval capabilities.

Note that this time index returns _ids, and we also add a tool for retrieving document by _id. The Verifier agent will use this functionality

In [4]:
from typing import Any, Dict, List, TypedDict, Optional

class SearchResult(TypedDict):
    """Represents a single search result entry."""
    start: int
    content: str
    video_id: str
    _id: int # added

class SearchTools:

    def __init__(self, index):
        self.index = index
        
    def search(self, query: str) -> List[SearchResult]:
        """
        Search the index for documents matching the given query.
    
        Args:
            query (str): The search query string.
    
        Returns:
            List[SearchResult]: A list of search results. Each result dictionary contains:
                - start (int): The starting position or offset within the source file.
                - content (str): A text excerpt or snippet containing the match.
                - video_id (str): Youtube video_id for the snippet.
                - _id (int): The unique id for the document
        """
        return self.index.search(
            query=query,
            num_results=5,
            output_ids=True,
        )

    def get_document_by_id(self, _id: int) -> Optional[SearchResult]:
        """
        Retrieve a document by its unique ID.

        Args:
            _id (int): The document id.

        Returns:
            SearchResult: The document corresponding to the given ID or None if it's not in the index.
        """
        if _id < 0 or _id >= len(self.index.docs):
            return None

        return self.index.docs[_id]

tools = SearchTools(index)


### Utility Classes and Functions

Helper class for tool call monitoring. We want to understand which agent is making the tool call, so we add the agent name:

In [5]:
from pydantic_ai.messages import FunctionToolCallEvent

class NamedCallback:

    def __init__(self, agent):
        self.agent_name = agent.name

    async def print_function_calls(self, ctx, event):
        # Detect nested streams
        if hasattr(event, "__aiter__"):
            async for sub in event:
                await self.print_function_calls(ctx, sub)
            return

        if isinstance(event, FunctionToolCallEvent):
            tool_name = event.part.tool_name
            args = event.part.args
            print(f"TOOL CALL ({self.agent_name}): {tool_name}({args})")

    async def __call__(self, ctx, event):
        return await self.print_function_calls(ctx, event)


## Clarifier Agent

Ask the user to clarify the user intent.

In [6]:
from pydantic import BaseModel, Field
from pydantic_ai import Agent

In [7]:
clarifier_instructions = """
Your task is to understand what user wants and what their intent is
Later this will be passed to the researcher to go deeper in exploring it

Use your own knowledge as well as the results from the search to clarify 
the intent of the user. 
ask the user for clarification once

after that, process the response and prepare the handoff to 
the research agent
"""

clarifier = Agent(
    name='clarifier_v2',
    instructions=clarifier_instructions,
    tools=[tools.search],
    model='gpt-4o-mini'
)


In [8]:
question = "do research on making money with AI"

results = await clarifier.run(
    user_prompt=question,
    event_stream_handler=NamedCallback(clarifier)
)

print(results.output)


TOOL CALL (clarifier_v2): search({"query":"making money with AI"})
It seems that you're interested in exploring various ways to make money with artificial intelligence (AI). From my search results, here are some key insights:

1. **Startup Funding and Investment**: Many AI startups engage with investors early on, often in pre-seed or seed rounds. Understanding how to make a compelling case for investment, such as detailing the need for funds for hiring or marketing, is essential (source: video excerpts discussing angel investors and VC funding).

2. **Open Source and B2B Software**: There's a significant focus on B2B software and open-source solutions that leverage AI. Companies often utilize community-driven go-to-market strategies and open-source projects as a viable business model, especially in Europe (source: discussions around investment in AI-based startups).

3. **Product Development**: Innovators are increasingly integrating AI into products in a way that doesn't require expli

After refining the instructions with ChatGPT

```
- refine the prompt and schema description
- I want the output contain the initial and refined information so it can be handed over to the agent 
```

In [9]:

clarifier_instructions = """
You are the CLARIFIER agent.

ROLE
Your job is to interpret and refine the user's research request so that it can be passed
to the RESEARCH agent for structured exploration.

OBJECTIVES
1. Understand what the user truly wants to learn or achieve (their intent).
2. Identify the core topic and any implicit goals (e.g., learn, compare, evaluate, predict, build).
3. Ask the user one targeted clarification question — to confirm scope, focus, or purpose.
4. Once the user responds, synthesize a refined version of their request that includes:
   - The clarified intent (what the user ultimately wants)
   - The initial request (in their own words)
   - The refined research focus (a precise version suitable for the RESEARCH agent)
   - 3–7 search queries that capture the clarified scope and intent
   - A short instruction summary for the RESEARCH agent explaining what to explore

DATA SOURCES
- You may use your own general knowledge to infer user intent.
- You may use the `search()` tool to quickly check ambiguous terms or context.

INTENT HANDLING
- Before searching, infer the underlying intent behind the user's request.
  Examples:
    - “getting into ML” → learning pathways, beginner resources, first projects
    - “AI safety concerns” → risks, ethical challenges, mitigation strategies
    - “startup funding trends” → investment patterns, valuations, stages
- Generate searches that reflect this **intent**, not just literal words.

CONSTRAINTS
- Ask the user for clarification **once only**.
- Do not fabricate information; if uncertain, clarify directly with the user.
- The goal is to output a structured handoff ready for the RESEARCH agent's Stage 1 process.
"""


In [10]:
class ResearchInstructions(BaseModel):
    """
    Output of the CLARIFIER agent.
    Provides both the user's raw input and the refined, structured guidance
    for the RESEARCH agent to begin its first stage.
    """
    initial_request: str = Field(
        ...,
        description="The user's original question or request, captured verbatim."
    )
    refined_request: str = Field(
        ...,
        description="A clarified, rephrased, and contextually grounded version of the initial request."
    )
    user_intent: str = Field(
        ...,
        description=(
            "A short summary (1–2 sentences) of what the user truly wants to accomplish "
            "or learn, inferred from both the initial request and clarification."
        )
    )
    queries: List[str] = Field(
        ...,
        description=(
            "A list of 3–7 specific search queries derived from the refined request, "
            "covering complementary angles or subtopics the RESEARCH agent should explore."
        )
    )
    instructions: str = Field(
        ...,
        description=(
            "Concise operational guidance for the RESEARCH agent, explaining how to use "
            "the queries and what to prioritize during Stage 1 research."
        )
    )


Test the clarifier with a sample interaction:

In [11]:
callback = NamedCallback(clarifier)

results = await clarifier.run(
    user_prompt='I want make money with ai',
    event_stream_handler=callback
)

print(results.output)

It sounds like you're interested in exploring opportunities to generate income using artificial intelligence. There are many avenues you can consider, such as starting a business, investing in AI-related stocks, developing AI applications, offering AI consulting services, or even engaging in affiliate marketing for AI products. 

Can you clarify what specific area or method you're most interested in? For example, are you looking for entrepreneurial ideas, investment opportunities, or ways to enhance your current job with AI?


In [13]:
results = await clarifier.run(
    user_prompt='I want to work as a freelancer',
    message_history=results.new_messages(),
    event_stream_handler=callback,
    output_type=ResearchInstructions
)

research_task = results.output

In [18]:
print("Initial request:", research_task.initial_request)
print("Refined request:", research_task.refined_request)
print("User intent:", research_task.user_intent)
print("Queries:", research_task.queries)
print("Instructions:", research_task.instructions)

Initial request: I want make money with ai
Refined request: I want to work as a freelancer in the field of artificial intelligence.
User intent: The user wants to explore freelance opportunities in artificial intelligence to generate income.
Queries: ['freelance AI jobs', 'top platforms for AI freelancers', 'AI freelance project ideas', 'how to get started as an AI freelancer', 'skills needed for freelance AI work', 'freelance AI pricing strategies']
Instructions: Research various opportunities for freelancers in AI, including job platforms, common project types, and essential skills. Provide insights on how to start in this field.


## Research Agent

Structured output:

In [19]:
class Reference(BaseModel):
    document_id: int
    quote: str
    timestamp: str

class ResearchKeyword(BaseModel):
    keyword: str
    relevant_references: List[Reference]

class VerifiableInsight(BaseModel):
    insight: str
    references: List[Reference]

class ResearchStageReport(BaseModel):
    stage: int
    explored_keywords: List[ResearchKeyword]
    verifiable_insights: List[VerifiableInsight]
    stage_summary: str
    recommended_next_steps: str
    recommended_next_keywords: List[str]

Instructions:

In [20]:
researcher_instructions = """
You are the RESEARCH agent.

ROLE
You perform structured research on a proprietary podcast/video database for a specific stage
of exploration (Stage 1, 2, or 3).

DATA SOURCE
- You may ONLY use the `search()` function, which returns transcript snippets with:
  { video_id, _id }
- Every reference must cite a real snippet with a valid `youtube_id`, `timestamp` and `_id`.
- Do not invent data, names, or timestamps.

STAGES

Stage 1 — Initial Search
- Use the user's question or clarified keywords from context.
- Identify 3–5 primary keywords, run one or more searches.
- Summarize the main findings, highlighting initial insights and directions.

Stage 2 — Expansion
- Build upon Stage 1 outputs (from context).
- Generate 5–7 related or complementary queries.
- Summarize recurring ideas and patterns across new results.

Stage 3 — Deep Dive
- Build upon Stage 1 and Stage 2.
- Generate 5–7 deeper or contrasting queries.
- Explore nuances, counterpoints, or mechanisms.
- Provide a more analytical synthesis.

CONSTRAINTS
- Use context from previous stages to guide deeper exploration.
- You must perform the necessary amount of queries for each stage:
    - 3-5 for stage 1
    - 5-7 for stage 2
    - 5-7 for stage 3
"""


Create the agent:

In [21]:
researcher = Agent(
    name='researcher_v2',
    instructions=researcher_instructions,
    tools=[tools.search],
    model='gpt-4o-mini',
    output_type=ResearchStageReport
)


Helper function to execute research stages:

In [22]:
async def do_research(
    stage: int,
    stage_instructions: str,
    previous_stages: List[ResearchStageReport]
) -> ResearchStageReport:
    previous_stages_json = '\n'.join([r.model_dump_json() for r in previous_stages])
    
    user_prompt = f"""
    Current stage: {stage}

    Stage instrustructions:
    {stage_instructions}

    Previous stages:
    {previous_stages_json}
    """

    callback = NamedCallback(researcher)
    
    results = await researcher.run(
        user_prompt=user_prompt,
        event_stream_handler=callback,
    )

    return results.output


In [23]:
stage_1_instructions = f"""
do initial research using this instructions:

{research_task.model_dump_json()}
"""

stage_1 = await do_research(
    stage=1,
    stage_instructions=stage_1_instructions,
    previous_stages=[]
)


TOOL CALL (researcher_v2): search({"query": "freelance AI jobs"})
TOOL CALL (researcher_v2): search({"query": "top platforms for AI freelancers"})
TOOL CALL (researcher_v2): search({"query": "AI freelance project ideas"})
TOOL CALL (researcher_v2): search({"query": "how to get started as an AI freelancer"})
TOOL CALL (researcher_v2): search({"query": "skills needed for freelance AI work"})
TOOL CALL (researcher_v2): search({"query": "freelance AI pricing strategies"})


This is what we get:

In [24]:
for kw in stage_1.explored_keywords:
    print(kw.keyword)
    for ref in kw.relevant_references:
        print(ref)
    print()


freelance AI jobs
document_id=491 quote="so again if somebody wants to I think you asked before it just won't start just go for it and it would be viable for sure you get some good skills" timestamp='49:00'
document_id=3256 quote='again online freelancing platforms such as upwork' timestamp='45:20'
document_id=3245 quote='there are these online freelancing platforms like upwork and if somebody needs some work so they kind of create a listing' timestamp='26:00'

top platforms for AI freelancers
document_id=28 quote='some favorite websites to find freelance jobs are Upwork, Toptal' timestamp='44:00'
document_id=29 quote='there are also public sector things your work you have to wear multiple hats' timestamp='46:57'
document_id=3256 quote='but usually like online freelancing platform such as Upwork' timestamp='20:20'

how to get started as an AI freelancer
document_id=4533 quote='to be an AI expert you at least need to know how to code...now you need to be able to talk or to spell' timest

After some refining with ChatGPT:

```
add field description for pydantic objects to: 
- make sure the keywords in the object match the actual tool use 
- add field in references to force it describe how reference is relevant to the keyword

Also, the selected quotes and insights must be useful for the user intention. How do I change that? 

```

Improved Research Agent:

In [25]:
from pydantic import BaseModel, Field
from typing import List

class Reference(BaseModel):
    """
    A single, verifiable citation to a transcript snippet or video segment.
    Must correspond to a real snippet returned by the `search()` tool.
    """
    document_id: int = Field(..., description="Internal ID of the transcript snippet.")
    quote: str = Field(..., description="Exact snippet that supports the keyword or insight.")
    timestamp: str = Field(..., description="Timestamp in the source video where the quote occurs, 'mm:ss' or 'h:mm:ss'")
    relevance_to_keyword: str = Field(..., description="Explanation of *how* this quote supports or illustrates the specific keyword or concept being explored.")
    relevance_to_user_intent:  str = Field(..., description="Explanation of *how* this quote help the user with their intent.")

class ResearchKeyword(BaseModel):
    """
    Represents a keyword explicitly searched during this research stage.
    Each keyword must match an actual query used in the search tool calls.
    """
    keyword: str = Field(..., description="The exact keyword or phrase used in the search() tool call.")
    relevant_references: List[Reference] = Field(
        ..., 
        description="List of transcript snippets directly relevant to this keyword. Each must include a 'relevance_to_keyword' explanation."
    )


class VerifiableInsight(BaseModel):
    """
    A synthesized insight that can be traced back to specific evidence.
    Each insight must be supported by at least one real reference.
    """
    insight: str = Field(..., description="An insight derived from the research, phrased in an evidence-based, verifiable way.")
    references: List[Reference] = Field(..., description="Citations that directly support this insight. Must contain valid timestamps and IDs.")


class ResearchStageReport(BaseModel):
    """
    Structured output for each research stage (1–3).
    Ensures traceability between searches, keywords, and findings.
    """
    stage: int = Field(..., description="The research stage number (1 = Initial Search, 2 = Expansion, 3 = Deep Dive).")
    explored_keywords: List[ResearchKeyword] = Field(
        ..., 
        description="List of the *exact* keywords used in this stage's search() calls, along with references showing their relevance."
    )
    verifiable_insights: List[VerifiableInsight] = Field(
        ..., 
        description="List of data-backed insights derived from the references gathered at this stage."
    )
    stage_summary: str = Field(..., description="Analytical summary of what was learned at this stage, connecting evidence to emerging themes.")
    recommended_next_steps: str = Field(..., description="Guidance for what to do in the next stage — e.g., new angles, counterpoints, or subtopics.")
    recommended_next_keywords: List[str] = Field(
        ..., 
        description="Suggested next queries based on gaps or promising directions discovered in this stage."
    )


In [26]:
researcher_instructions = """
You are the RESEARCH agent.

ROLE
You perform structured research on a proprietary podcast/video database for a specific stage
of exploration (Stage 1, 2, or 3).

DATA SOURCE
- You may ONLY use the `search()` function
- Every reference must cite a real snippet with a valid `youtube_id`, `timestamp` and `_id`.
- Do not invent data, names, or timestamps.

STAGES

Stage 1 — Initial Search
- Use the user’s question or clarified keywords from context.
- Identify 3–5 primary keywords, run one or more searches.
- Summarize the main findings, highlighting initial insights and directions.

Stage 2 — Expansion
- Build upon Stage 1 outputs (from context).
- Generate 5–7 related or complementary queries.
- Summarize recurring ideas and patterns across new results.

Stage 3 — Deep Dive
- Build upon Stage 1 and Stage 2.
- Generate 5–7 deeper or contrasting queries.
- Explore nuances, counterpoints, or mechanisms.
- Provide a more analytical synthesis.

CONSTRAINTS
- Use context from previous stages to guide deeper exploration.
- You must perform the necessary amount of queries for each stage:
    - 3-5 for stage 1
    - 5-7 for stage 2
    - 5-7 for stage 3
"""

researcher = Agent(
    name='researcher_v2',
    instructions=researcher_instructions,
    tools=[tools.search],
    model='gpt-4o-mini',
    output_type=ResearchStageReport
)


In [27]:
stage_1 = await do_research(
    stage=1,
    stage_instructions=stage_1_instructions,
    previous_stages=[]
)

for kw in stage_1.explored_keywords:
    print(kw.keyword)
    for ref in kw.relevant_references:
        print(ref)
    print()

for insight in stage_1.verifiable_insights:
    print(insight)


TOOL CALL (researcher_v2): search({"query": "freelance AI jobs"})
TOOL CALL (researcher_v2): search({"query": "top platforms for AI freelancers"})
TOOL CALL (researcher_v2): search({"query": "AI freelance project ideas"})
TOOL CALL (researcher_v2): search({"query": "how to get started as an AI freelancer"})
TOOL CALL (researcher_v2): search({"query": "skills needed for freelance AI work"})
TOOL CALL (researcher_v2): search({"query": "freelance AI pricing strategies"})
freelance AI jobs
document_id=28 quote='I think it can be a good idea while you are... searching and for you to get a job so doing consistent and persistent way of uh...' timestamp='48:50' relevance_to_keyword='This quote emphasizes the importance of persistence in securing freelance jobs in AI and highlights a common struggle among freelancers.' relevance_to_user_intent="This supports the user's intent of exploring freelance opportunities, as it offers insight into what steps to take."
document_id=29 quote='How do you go

Continue with Stage 2 and 3

In [28]:
stage_2 = await do_research(
    stage=2,
    stage_instructions="continue research",
    previous_stages=[stage_1]
)

stage_3 = await do_research(
    stage=3,
    stage_instructions="continue research, go deeper and broader, explore tangently relaveted topics",
    previous_stages=[stage_1, stage_2]
)


TOOL CALL (researcher_v2): search({"query": "best AI project types for freelancers"})
TOOL CALL (researcher_v2): search({"query": "most effective marketing strategies for AI freelancers"})
TOOL CALL (researcher_v2): search({"query": "success stories of AI freelancers"})
TOOL CALL (researcher_v2): search({"query": "freelance AI community building"})
TOOL CALL (researcher_v2): search({"query": "AI freelancing niches"})
TOOL CALL (researcher_v2): search({"query": "tools for AI freelancers"})
TOOL CALL (researcher_v2): search({"query": "freelance AI contract tips"})
TOOL CALL (researcher_v2): search({"query": "niche markets for AI freelancers"})
TOOL CALL (researcher_v2): search({"query": "partnership strategies for AI freelancers"})
TOOL CALL (researcher_v2): search({"query": "impact of freelance platforms on earnings"})
TOOL CALL (researcher_v2): search({"query": "best practices for successful freelancing"})
TOOL CALL (researcher_v2): search({"query": "legal considerations in freelance c

## Synthesizer and Verifier Agent

Instructions:

In [29]:
synthesizer_instructions = """
You synthesize research findings from all three stages (StageReports 1–3)
into a cohesive, factual final report.

TASKS
1. Read and interpret all reports
2. Verify each claim you put in the article
3. Make sure the output matches the intention of the user
4. Create the article

ARTICLE RULES
- The article should have instruction, 5-6 sections and conclusion 
- Each section should group 3–4 related claims.
- Each claim: 3–4 sentences and reference
- Do not add new facts beyond what's supported in reports
- You must verify each source
"""

synthesizer = Agent(
    name='synthesizer',
    instructions=synthesizer_instructions,
    tools=[tools.get_document_by_id],
    model='gpt-4o-mini',
)   


Test the basic version:

In [30]:
all_reports = [stage_1, stage_2, stage_3]
reports = '\n'.join([r.model_dump_json() for r in all_reports])

user_prompt = f"""
initial request:
{research_task.model_dump_json()}

reports:
{reports}
"""

callback = NamedCallback(synthesizer)

results = await synthesizer.run(
    user_prompt=user_prompt,
    event_stream_handler=callback
)


TOOL CALL (synthesizer): get_document_by_id({"_id": 28})
TOOL CALL (synthesizer): get_document_by_id({"_id": 29})
TOOL CALL (synthesizer): get_document_by_id({"_id": 1960})
TOOL CALL (synthesizer): get_document_by_id({"_id": 1958})
TOOL CALL (synthesizer): get_document_by_id({"_id": 3244})
TOOL CALL (synthesizer): get_document_by_id({"_id": 3256})
TOOL CALL (synthesizer): get_document_by_id({"_id": 1091})
TOOL CALL (synthesizer): get_document_by_id({"_id": 3405})
TOOL CALL (synthesizer): get_document_by_id({"_id": 3259})
TOOL CALL (synthesizer): get_document_by_id({"_id": 7078})
TOOL CALL (synthesizer): get_document_by_id({"_id": 3422})


In [31]:
print(results.output)

# Freelancing in Artificial Intelligence: A Comprehensive Guide

Freelancing in the field of Artificial Intelligence (AI) has become a lucrative career choice for many professionals looking to leverage their skills. This article provides a structured approach for potential AI freelancers, detailing the essential steps to navigate this dynamic landscape. 

## 1. Understanding Freelance AI Opportunities

Freelancing opportunities in AI range from machine learning and data analysis to natural language processing and computer vision projects. Freelancers can find work through **online platforms** like Upwork, Freelancer, and others that connect them with clients in need of specialized expertise. According to a source, platforms such as Upwork allow freelancers to apply directly to project listings posted by clients, making it easier to find opportunities tailored to their skills (Video 1, Timestamp: 25:42) and (Stage 1 Report). Persistence and continuous improvement of one's freelancer pro

Improved agent:

In [32]:
synthesizer_instructions = """
You are the SYNTHESIZER agent.

ROLE
You create a cohesive, factual article by synthesizing verified information from all
three research stages (StageReports 1–3).

DATA SOURCES
- You will receive one or more `ResearchStageReport` objects, each containing
  verifiable references with document_ids, timestamps, and quotes.
- You have access to the tool `get_document_by_id` to retrieve full source text
  for any reference.
- You must use this tool to verify every claim that appears in your article.

TASKS
1. Carefully read all StageReports and extract recurring insights and verified facts.
2. Use `get_document_by_id` to check each cited reference and confirm that
   the quote or insight is correctly represented.
3. Only include claims that are explicitly supported by at least one verified source.
4. Synthesize related findings into 5–6 cohesive sections with a logical flow.
5. Ensure that the article aligns with the original user intent (as passed from the clarifier).

ARTICLE STRUCTURE
- Introduction: Summarize what the article will explore and why it matters.
- 5-6 body sections, each:
  - Centered on one major theme or subtopic.
  - Contains 3–4 related claims (each 3–4 sentences long).
  - Each claim includes an in-text reference
- Conclusion: Summarize the most important insights and actionable takeaways.

VERIFICATION RULES
- For every claim, retrieve at least one cited source using `get_document_by_id`
  and confirm that the text supports the claim.
- If a reference cannot be verified or is inconsistent, omit it.
- Do not invent or infer facts beyond what’s supported by verified material.

STYLE
- Maintain factual, neutral, and coherent tone.
- Avoid speculation, exaggeration, or unsupported synthesis.
- Write in clear prose suitable for an informed but general audience.

OUTPUT
- A single, well-structured factual article ready for presentation.
- All references cited
""".strip()

synthesizer = Agent(
    name='synthesizer_v2',
    instructions=synthesizer_instructions,
    tools=[tools.get_document_by_id],
    model='gpt-4o-mini',
)   


In [ ]:
all_reports = [stage_1, stage_2, stage_3]
reports = '\n'.join([r.model_dump_json() for r in all_reports])

user_prompt = f"""
initial request:
{research_task.model_dump_json()}

reports:
{reports}
"""

callback = NamedCallback(synthesizer)

results = await synthesizer.run(
    user_prompt=user_prompt,
    event_stream_handler=callback.print_function_calls
)

# Checks


TOOL CALL (synthesizer_v2): get_document_by_id({"_id": 28})
TOOL CALL (synthesizer_v2): get_document_by_id({"_id": 29})
TOOL CALL (synthesizer_v2): get_document_by_id({"_id": 3256})
TOOL CALL (synthesizer_v2): get_document_by_id({"_id": 1960})
TOOL CALL (synthesizer_v2): get_document_by_id({"_id": 1958})
TOOL CALL (synthesizer_v2): get_document_by_id({"_id": 3244})
TOOL CALL (synthesizer_v2): get_document_by_id({"_id": 1091})
TOOL CALL (synthesizer_v2): get_document_by_id({"_id": 3405})
TOOL CALL (synthesizer_v2): get_document_by_id({"_id": 3259})
TOOL CALL (synthesizer_v2): get_document_by_id({"_id": 7078})
TOOL CALL (synthesizer_v2): get_document_by_id({"_id": 3422})


Results:

In [34]:
print(results.output)

# Exploring Freelance Opportunities in Artificial Intelligence

In today's digital economy, freelancing in artificial intelligence (AI) presents an exciting avenue for individuals eager to leverage their skills for profit. This article will delve into key areas of interest for aspiring AI freelancers, including how to get started, the necessary skills, top platforms for finding work, lucrative project ideas, and effective pricing strategies. Understanding these components will empower freelancers to navigate the AI landscape and maximize their earning potential.

## Getting Started in AI Freelancing

Embarking on a freelancing career in AI can be incredibly rewarding but comes with its own challenges. A particularly effective strategy for entering the freelance market is actively promoting oneself through a well-prepared CV. For instance, one individual shared their experience of posting their CV online, leading to their first client just one week later (Document ID: 1960). This method